In [5]:
import numpy as np
import pandas as pd

#  Profil für Anzahl der Zeitschritte
df_profil = pd.read_excel("profil_winter.xlsx")

########################################part 1#####################################
load_bus = pd.read_excel('load_bus_winter.xlsx')

load_profil = pd.DataFrame(columns=['t'+str(i) for i in range(1,97)])
#load_profil['b'] = load_bus['b']
for i in range(1,97):
    load_profil['t'+str(i)] = df_profil['profil'][i-1] * load_bus['Active demand']
    
load_profil=load_profil.transpose()

load_profil=load_profil.rename(columns={i: i+1 for i in range(0, 32)})

########################################part 2#####################################
#erzeugen der Prognosefehler

# Anzahl der Zufallsvariablen pro Zeitpunkt
num_variables = 32

# Anzahl der Szenarien
num_scenarios = 100

# Standardabweichung der normalverteilten Zufallsvariablen
std_dev = 0.005

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(num_scenarios):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und jeden Zeitpunkt
    random_variables = np.random.normal(0, std_dev, (len(df_profil), num_variables))
    
    # Berechne die Summe der Zufallsvariablen für jeden Zeitpunkt
    sum_random_variables = np.sum(random_variables, axis=1)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / num_variables

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor.reshape(-1, 1)

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen für jeden Zeitpunkt innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=1), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen ein
    df_varied = pd.DataFrame(random_variables_adjusted)
    
    # Füge eine Spalte für den Slack bus hinzu
    df_varied.insert(0, '1',  0)

    # Füge eine Spalte für den Zeitpunkt hinzu
    df_varied.insert(0, "t", range(1, len(df_profil)+1))
    
    # Benenne die Spalten von 1 bis 33
    df_varied.columns = ["t", 1] + list(range(2, num_variables+2))
    
    new_varied = (df_varied.T + load_profil.T).T  - load_profil  
    
    # Füge das DataFrame mit den angepassten Zufallsvariablen dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], ignore_index=True)
    
    new_results = (results + load_profil) - load_profil  

    
# Füge eine Spalte für die Szenarien hinzu
new_results.insert(0, "Sc", np.repeat(np.arange(1, num_scenarios+1), len(df_profil)))

# Sortiere die Ergebnisse nach Szenario und Zeitpunkt
new_results = new_results.sort_values(by=["Sc", "t"])

# Speichere das DataFrame der Ergebnisse in einer CSV-Datei
new_results.to_excel("Ergebnisse.xlsx", index=False)




AssertionError: Gaps in blk ref_locs